# T1486 - Data Encrypted for Impact
Adversaries may encrypt data on target systems or on large numbers of systems in a network to interrupt availability to system and network resources. They can attempt to render stored data inaccessible by encrypting files or data on local and remote drives and withholding access to a decryption key. This may be done in order to extract monetary compensation from a victim in exchange for decryption or a decryption key (ransomware) or to render data permanently inaccessible in cases where the key is not saved or transmitted.(Citation: US-CERT Ransomware 2016)(Citation: FireEye WannaCry 2017)(Citation: US-CERT NotPetya 2017)(Citation: US-CERT SamSam 2018)

In the case of ransomware, it is typical that common user files like Office documents, PDFs, images, videos, audio, text, and source code files will be encrypted (and often renamed and/or tagged with specific file markers). Adversaries may need to first employ other behaviors, such as [File and Directory Permissions Modification](https://attack.mitre.org/techniques/T1222) or [System Shutdown/Reboot](https://attack.mitre.org/techniques/T1529), in order to unlock and/or gain access to manipulate these files.(Citation: CarbonBlack Conti July 2020) In some cases, adversaries may encrypt critical system files, disk partitions, and the MBR.(Citation: US-CERT NotPetya 2017) 

To maximize impact on the target organization, malware designed for encrypting data may have worm-like features to propagate across a network by leveraging other attack techniques like [Valid Accounts](https://attack.mitre.org/techniques/T1078), [OS Credential Dumping](https://attack.mitre.org/techniques/T1003), and [SMB/Windows Admin Shares](https://attack.mitre.org/techniques/T1021/002).(Citation: FireEye WannaCry 2017)(Citation: US-CERT NotPetya 2017) Encryption malware may also leverage [Internal Defacement](https://attack.mitre.org/techniques/T1491/001), such as changing victim wallpapers, or otherwise intimidate victims by sending ransom notes or other messages to connected printers (known as "print bombing").(Citation: NHS Digital Egregor Nov 2020)

In cloud environments, storage objects within compromised accounts may also be encrypted.(Citation: Rhino S3 Ransomware Part 1)

## Atomic Tests

### Atomic Test #1 - Encrypt files using gpg (Linux)
Uses gpg to encrypt a file

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Finds where gpg is located

##### Check Prereq Commands:
```bash
which_gpg=`which gpg`

```
##### Get Prereq Commands:
```bash
(which yum && yum -y install epel-release gpg)||(which apt-get && DEBIAN_FRONTEND=noninteractive apt-get install -y gpg)

```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
echo "passwd" | $which_gpg --batch --yes --passphrase-fd 0 --cipher-algo AES-256 -o /tmp/passwd.gpg -c /etc/passwd
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 1

#### Cleanup: 
```bash
rm /tmp/passwd.gpg
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Encrypt files using 7z (Linux)
Uses 7z to encrypt a file

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Finds where 7z is located

##### Check Prereq Commands:
```bash
which_7z=`which 7z`

```
##### Get Prereq Commands:
```bash
None
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
$which_7z a -ppasswd /tmp/passwd.zip /etc/passwd
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 2

#### Cleanup: 
```bash
$which_7z e /tmp/passwd.zip
rm /tmp/passwd.zip
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Encrypt files using ccrypt (Linux)
Attempts to encrypt data on target systems as root to simulate an inturruption authentication to target system. If root permissions are not available then attempts to encrypt data within user's home directory.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Finds where ccencrypt and ccdecrypt is located and copies input file

##### Check Prereq Commands:
```bash
which_ccencrypt=`which ccencrypt`
which_ccdecrypt=`which ccdecrypt`
if [[ $USER == "root" ]]; then cp /etc/passwd /tmp/passwd; else cp ~/.bash_history /tmp/passwd; fi

```
##### Get Prereq Commands:
```bash
(which yum && yum -y install epel-release ccrypt)||(which apt-get && DEBIAN_FRONTEND=noninteractive apt-get install -y ccrypt)

```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
if [[ $USER == "root" ]]; then $which_ccencrypt /etc/passwd; file /etc/passwd.cpt; sudo su; else $which_ccencrypt ~/.bash_history; file ~/.bash_history.cpt; sudo su; fi
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 3

#### Cleanup: 
```bash
if [[ $USER == "root" ]]; then mv /tmp/passwd /etc/passwd; else cp /tmp/passwd ~/.bash_history; fi 
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Encrypt files using openssl (Linux)
Uses openssl to encrypt a file

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Finds where openssl is located

##### Check Prereq Commands:
```bash
which_openssl=`which openssl`

```
##### Get Prereq Commands:
```bash
None
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
$which_openssl genrsa -out /tmp/key.pem 2048
$which_openssl rsa -in /tmp/key.pem -pubout -out /tmp/pub.pem
$which_openssl rsautl -encrypt -inkey /tmp/pub.pem -pubin -in /etc/passwd -out /tmp/passwd.zip
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 4

#### Cleanup: 
```bash
$which_openssl rsautl -decrypt -inkey /tmp/key.pem -in /tmp/passwd.zip
rm /tmp/passwd.zip
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 4 -Cleanup

### Atomic Test #5 - PureLocker Ransom Notebuilding the IOC (YOUR_FILES.txt) for the PureLocker ransomware 
https://www.bleepingcomputer.com/news/security/purelocker-ransomware-can-lock-files-on-windows-linux-and-macos/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
echo T1486 - Purelocker Ransom Note > %USERPROFILE%\Desktop\YOUR_FILES.txt
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 5

#### Cleanup: 
```cmd
del %USERPROFILE%\Desktop\YOUR_FILES.txt >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1486 -TestNumbers 5 -Cleanup

## Detection
Use process monitoring to monitor the execution and command line parameters of binaries involved in data destruction activity, such as vssadmin, wbadmin, and bcdedit. Monitor for the creation of suspicious files as well as unusual file modification activity. In particular, look for large quantities of file modifications in user directories.

In some cases, monitoring for unusual kernel driver installation activity can aid in detection.

In cloud environments, monitor for events that indicate storage objects have been anomalously replaced by copies.


## Shield Active Defense
### System Activity Monitoring 
 Collect system activity logs which can reveal adversary activity. 

 Capturing system logs can show logins, user and system events, etc.  Collecting this data and potentially sending it to a centralized location can help reveal the presence of an adversary and the actions they perform on a compromised system.
#### Opportunity
There is an opportunity to create a detection with a moderately high probability of success.
#### Use Case
A defender can use process monitoring to look for the execution of utilities commonly used for ransomware and other data encryption.
#### Procedures
Ensure that systems capture and retain common system level activity artifacts that might be produced.
Monitor Windows systems for event codes that reflect an adversary changing passwords, adding accounts to groups, etc.
